# Assignment-1: Transformations and representations

Team Name: GroveStreet

Roll number: 2021701023, 2021900013

# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `Set Up` for detailed step-by-step instructions about the installation setup.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. 
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 11/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.


# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

# Set Up

We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. All assignments will be python based, hence familiarising yourself with Python is essential.


## Setting up Anaconda environment (Recommended)

1. Install Anaconda or Miniconda from [here](https://docs.conda.io/projects/conda/en/latest/user-guide/install/linux.html) depending on your requirements.
2. Now simply run `conda env create -f environment.yml` in the current folder to create an environment `mr_assignment1` (`environment.yml` can be found in `misc/`).
3. Activate it using `conda activate mr_assignment1`.

## Setting up Virtual environment using venv

You can also set up a virtual environment using venv

1. Run `sudo apt-get install python3-venv` from command line.
2. `python3 -m venv ~/virtual_env/mr_assignment1`. (you can set the environment path to anything)
3. `source ~/virtual_env/mr_assignment1/bin/activate`
4. `pip3 install -r requirements.txt` from the current folder (`requirements.txt` can be found in `misc/`).

In [1]:
import time
from pprint import pprint
import open3d as o3d

import numpy as np
from scipy.optimize import fsolve
from scipy.spatial.transform import Rotation
from transforms3d.euler import euler2mat, mat2euler

import torch
import pytorch3d.transforms as transforms

np.set_printoptions(precision=3, suppress=True)

# 1. Getting started with Open3D

Open3D is an open-source library that deals with 3D data, such as point clouds, mesh. We'll be using Open3D frequently as we work with point clouds. Let's start with something simple:

<img src="misc/bunny.jpg" alt="drawing" width="200"/>

1. Read the Stanford Bunny file (in `data/`) given to you and visualise it using Open3D.
2. Convert the mesh to a point cloud and change the colour of points.
3. Set a predefined viewing angle (using Open3D) for visualization and display the axes while plotting.
4. Scale, Transform, and Rotate the rabbit (visualise after each step).
5. Save the point cloud as bunny.pcd.

In [2]:
# Reading Data
mesh = o3d.io.read_triangle_mesh('./data/bunny.ply')
o3d.visualization.draw([mesh])

## Convert Mesh to Point Cloud & Change Color
point_cloud = mesh.sample_points_poisson_disk(5000)
point_cloud.paint_uniform_color([1, 0.5, 0.])
o3d.visualization.draw([point_cloud])

## Show Axis
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.03, origin=np.array([0., 0., 0.]))
o3d.visualization.draw([point_cloud, mesh_frame], 
                       lookat=np.array([[.0], [.1], [.0]], dtype=np.float64),
                       eye=np.array([[0.0], [0.0], [-0.4]], dtype=np.float64),
                       up=np.array([[0.0], [0.5], [0.0]], dtype=np.float64),
                      )

## Scale Point Cloud by 0.5
point_cloud.scale(scale=0.5, center=np.array([[.0], [.0], [.0]], dtype=np.float64))
o3d.visualization.draw([point_cloud, mesh_frame],
                       lookat=np.array([[.0], [.1], [.0]], dtype=np.float64),
                       eye=np.array([[0.0], [0.0], [-0.4]], dtype=np.float64),
                       up=np.array([[0.0], [0.5], [0.0]], dtype=np.float64),
                      )

## Transform Point Cloud (Rotate about Y axis, translate to [0.2, 0, 0])
point_cloud.transform(np.array([[0.707, 0.0, 0.707, 0.2],
                                [0.0, 1.0, 0.0, 0.0],
                                [-0.707, 0.0, 0.707, 0.0],
                                [0.0, 0.0, 0.0, 1.0]], dtype=np.float64))
o3d.visualization.draw([point_cloud, mesh_frame],
                       lookat=np.array([[.0], [.1], [.0]], dtype=np.float64),
                       eye=np.array([[0.0], [0.0], [-0.4]], dtype=np.float64),
                       up=np.array([[0.0], [0.5], [0.0]], dtype=np.float64),
                      )

## Rotate Point Cloud (Rotate about Y axis)
point_cloud.rotate(R=np.array([[0.707, 0.0, 0.707],
                                [0.0, 1.0, 0.0],
                                [-0.707, 0.0, 0.707]], dtype=np.float64))
o3d.visualization.draw([point_cloud, mesh_frame],
                       lookat=np.array([[.0], [.1], [.0]], dtype=np.float64),
                       eye=np.array([[0.0], [0.0], [-0.4]], dtype=np.float64),
                       up=np.array([[0.0], [0.5], [0.0]], dtype=np.float64),
                      )

## Save 
o3d.io.write_point_cloud('./data/bunny.pcd', point_cloud)

True

# 2. Transformations and representations

## a) Euler angles
1. Write a function that returns a rotation matrix given the angles $\alpha$, $\beta$, and $\gamma$ in radians (X-Y-Z)

2. Solve for angles using ```fsolve from scipy``` for three initializations of your choice and compare.
$$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] 
$$

$$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] 
$$

3. What is a Gimbal lock? 

4. Show an example where a Gimbal lock occurs and visualize the Gimbal lock on the given bunny point cloud. You have to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*


<font color='blue' size=4.0/> 2.a.1 Function that returns a rotation matrix given the angles $\alpha$, $\beta$, and $\gamma$ in radians (X-Y-Z)

In [3]:
## Order can be reverse, Since X-Y-Z angles
def get_rotation_matrix(alpha, beta, gamma):
    
    r_alpha = np.matrix([[1.0, 0.0, 0.0],
                        [0.0, np.cos(alpha), -np.sin(alpha)],
                        [0.0, np.sin(alpha), np.cos(alpha)]], dtype=np.float64)
    r_alpha = np.around(r_alpha, decimals=4)

    r_beta = np.matrix([[np.cos(beta), 0.0, np.sin(beta)],
                        [0.0, 1.0, 0.0],
                        [-np.sin(beta), 0.0, np.cos(beta)]], dtype=np.float64)
    r_beta = np.around(r_beta, decimals=4)

    r_gamma = np.matrix([[np.cos(gamma), -np.sin(gamma), 0.0],
                        [np.sin(gamma), np.cos(gamma), 0.0],
                        [0.0, 0.0, 1.0]], dtype=np.float64)
    r_gamma = np.around(r_gamma, decimals=3)
    
    return r_alpha @ r_beta @ r_gamma

R = get_rotation_matrix(0, 0, 0)
print(R)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


<font color='blue' size=4.0/> 2.a.2 Solve for angles using `fsolve from scipy` for three initializations of your choice and compare

In [4]:
M = np.array([[0.26200263, -0.19674724, 0.944799],
              [0.21984631, 0.96542533, 0.14007684],
              [-0.93969262, 0.17101007, 0.29619813]])

N = np.array([[0., -0.173648178, 0.984807753],
              [0., 0.984807753, 0.173648178],
              [-1., 0., 0.]])

In [5]:
def get_euler_from_rotation(x, R):
    return [np.cos(x[0])*np.cos(x[1]) - R[2, 2], np.sin(x[1]) - R[0, 2], np.cos(x[1])*np.cos(x[2]) - R[0, 0]]

In [6]:
x_M = [0.2, 0.1, np.pi/2]
x_M = fsolve(get_euler_from_rotation, x_M, args=M)
print(f"Euler Angles for matrix M using fsolve: {x_M}")

Euler Angles for matrix M using fsolve: [-0.442  1.237  0.644]


In [7]:
x_N = [0.2, 0.1, np.pi/2]
x_N = fsolve(get_euler_from_rotation, x_N, args=N)
print(f"Euler Angles for matrix N using fsolve: {x_N}")

Euler Angles for matrix N using fsolve: [-1.571  1.396  1.571]


In [8]:
## Verification
R_M = get_rotation_matrix(*x_M)
print("Reconstructed matrix M from predcited angles \n", R_M)

R_N = get_rotation_matrix(*x_N)
print("Reconstructed matrix N from predcited angles \n", R_N)

Reconstructed matrix M from predcited angles 
 [[ 0.262 -0.197  0.945]
 [ 0.219  0.966  0.14 ]
 [-0.94   0.17   0.296]]
Reconstructed matrix N from predcited angles 
 [[ 0.    -0.174  0.985]
 [ 0.     0.985  0.174]
 [-1.     0.     0.   ]]


In [9]:
## Verify (Closed Form)
def rotationToEuler(R):
    return [np.arctan2(-R[1, 2], R[2, 2]), np.arctan2(R[0, 2], np.sqrt(1 - R[0, 2]**2)), np.arctan2(-R[0, 1], R[0, 0])]

print(f'Closed Form Euler Angles for matrix M: {rotationToEuler(M)}')
print(f'Closed Form Euler Angles for matrix N: {rotationToEuler(N)}')

Closed Form Euler Angles for matrix M: [-0.44174662088223243, 1.2369806006375788, 0.6440999603368792]
Closed Form Euler Angles for matrix N: [-1.5707963267948966, 1.3962634015251603, 1.5707963267948966]


<font color='blue' size=4.0/> 2.a.3 What is a Gimbal lock?

Gimbal lock occurs when two of the three axis of rotations become parallel to each other resulting in loss of one degree of freedom and restricting rotation to two dimensional space. 

<font color='blue' size=4.0/> 2.a.4 Example of Gimbal lock and visualize the Gimbal lock on the given bunny point cloud

In [10]:
## Gimbals
torus_x = o3d.geometry.TriangleMesh.create_torus(torus_radius=2.,tube_radius=0.1)
torus_x.paint_uniform_color([1., 0., 0.])
R = euler2mat(np.pi/2, 0., 0., 'sxyz')
torus_x.rotate(R=R)

torus_y = o3d.geometry.TriangleMesh.create_torus(torus_radius=1.5, tube_radius=0.1)
torus_y.paint_uniform_color([0., 1., 0.])
R = euler2mat(0., np.pi/2, 0., 'sxyz')
torus_y.rotate(R=R)

torus_z = o3d.geometry.TriangleMesh.create_torus(tube_radius=0.1)
torus_z.paint_uniform_color([0., 0., 1.])
R = euler2mat(0., 0., np.pi/2, 'sxyz')
torus_z.rotate(R=R)

## Gimbal Lock Animation
point_cloud = o3d.io.read_point_cloud('./data/bunny.ply')
point_cloud.scale(scale=20., center=np.array([[.0], [.0], [.0]], dtype=np.float64))
point_cloud.translate(translation=np.array([0., 2., 0.]))

vis = o3d.visualization.Visualizer()
vis.create_window()

vis.add_geometry(point_cloud)

vis.add_geometry(torus_x)
vis.add_geometry(torus_y)
vis.add_geometry(torus_z)

step = 10
axes = [("y", 90), ("z", 90), ("x", 90), ("z", 90), ("x", 90), ("y", 90)]
for ax, degree in axes:
    angle = 0
    while angle < degree:
        time.sleep(0.7)
        
        if ax == "x":
            R = get_rotation_matrix(np.radians(step), 0., 0.)
            torus_x.rotate(R=R)
            torus_y.rotate(R=R)
            torus_z.rotate(R=R)
            vis.update_geometry(torus_x)
            vis.update_geometry(torus_y)
            vis.update_geometry(torus_z)
        elif ax == "y":
            R = get_rotation_matrix(0., np.radians(step), 0.)
            torus_y.rotate(R=R)
            torus_z.rotate(R=R)
            vis.update_geometry(torus_y)
            vis.update_geometry(torus_z)
        elif ax == "z":
            R = get_rotation_matrix(0., 0., np.radians(step))
            torus_z.rotate(R=R)
            vis.update_geometry(torus_z)
            
        point_cloud.rotate(R=R)

        angle += step
        vis.update_geometry(point_cloud)
        
        vis.poll_events()
        vis.update_renderer()
    
vis.destroy_window()

## b) Quaternions

1. What makes Quaternions popular in graphics? 
2. Convert a rotation matrix to quaternion and vice versa. Do not use inbuilt libraries for this question.
3. Perform matrix multiplication of two $\mathcal{R}_{3 \times 3}$ rotation matrices and perform the same transformation in the quaternion space. Verify if the final transformation obtained in both the cases are the same.
4. Try to interpolate any 3D model (cube / bunny / not sphere obviously!!) between two rotation matrices and visualize!

The above questions require you to **code your own functions** and **only verify** using inbuilt functions.

<font color='blue' size=4.0/> 2.b.1 What makes Quaternions popular in graphics?

Quaternions allow rotations about multiple axis simultaenously and hence do not suffer from gumbel lock that occurs during sequential rotations about individual axes. Additionally, Quaternions consume less memory and are faster to compute than rotation matrices. Interpolating between rotations is easier to express in terms of quaternion compared to 3x3 rotation matrices. Quaternion are naturally preferred language to preserve angle when a curved 3D texture or surface is mapped to a image i.e; when we try to flatten the curved surface the angles seen in the 3D surface needs to be preserved.

<font color='blue' size=4.0/> 2.b.2 Convert a rotation matrix to quaternion and vice versa.

In [11]:
def quaternion_to_rotation(q):
    return np.array([[q[0]**2 + q[1]**2 - q[2]**2 - q[3]**2, 2*(q[1]*q[2] - q[0]*q[3]), 2*(q[1]*q[3] + q[0]*q[2])],
                     [2*(q[1]*q[2] + q[0]*q[3]), q[0]**2 - q[1]**2 + q[2]**2 - q[3]**2, 2*(q[2]*q[3] - q[0]*q[1])],
                     [2*(q[1]*q[3] - q[0]*q[2]), 2*(q[2]*q[3] + q[0]*q[1]), q[0]**2 - q[1]**2 - q[2]**2 + q[3]**2]])

def quaternion_conjugate(q):
    return q*np.array([1, -1, -1, -1])

def rotation_to_quaternion(R):
    tr = R[0, 0] + R[1, 1] + R[2, 2]
    return np.array([0.5 * np.sqrt(1 + tr), 
            np.copysign(0.5 * np.sqrt(1 + R[0,0] - R[1,1] - R[2,2]), R[2, 1] - R[1, 2]),
            np.copysign(0.5 * np.sqrt(1 - R[0,0] + R[1,1] - R[2,2]), R[0, 2] - R[2, 0]),
            np.copysign(0.5 * np.sqrt(1 - R[0,0] - R[1,1] + R[2,2]), R[1, 0] - R[0, 1])])

In [12]:
rot_mat = M
print("Original Matrix: \n", rot_mat)

Original Matrix: 
 [[ 0.262 -0.197  0.945]
 [ 0.22   0.965  0.14 ]
 [-0.94   0.171  0.296]]


In [13]:
## Convert Rotation Matrix to Quaternion
quaternion = rotation_to_quaternion(rot_mat)
print(f"Quaternion from matrix: {quaternion}")

Quaternion from matrix: [0.794 0.01  0.593 0.131]


In [14]:
## Convert Quaternion to Rotation Matrix
rotation_matrix = quaternion_to_rotation(quaternion)
print("Rotation Matrix from Quaternion: \n", np.around(rotation_matrix, decimals=6))

Rotation Matrix from Quaternion: 
 [[ 0.262 -0.197  0.945]
 [ 0.22   0.965  0.14 ]
 [-0.94   0.171  0.296]]


In [15]:
## Verification for Quaternion from Rotation Matrix
r = Rotation.from_matrix(rot_mat)
scipy_quat = r.as_quat()[[-1, 0, 1, 2]]
print("Scipy Result: ", scipy_quat)

Scipy Result:  [0.794 0.01  0.593 0.131]


In [16]:
## Verification for Quaternion to Rotation Matrix
r = Rotation.from_quat(scipy_quat[[1, 2, 3, 0]])
print("Scipy Result: \n", np.around(r.as_matrix(), decimals=6))

Scipy Result: 
 [[ 0.262 -0.197  0.945]
 [ 0.22   0.965  0.14 ]
 [-0.94   0.171  0.296]]


<font color='blue' size=4.0/> 2.b.3 Perform matrix multiplication of two rotation matrices and perform the same transformation in the quaternion space

In [17]:
def multiply_quaternions(a, b):
    O = np.outer(a, b)
    return np.array([O[0, 0] - O[1, 1] - O[2, 2] - O[3, 3], 
                     O[0, 1] + O[1, 0] + O[2, 3] - O[3, 2],
                     O[0, 2] + O[2, 0] - O[1, 3] + O[3, 1],
                     O[0, 3] + O[3, 0] + O[1, 2] - O[2, 1]])

def normalize(x):
    norm = np.linalg.norm(x, 2)
    x = x / norm
    return x

In [18]:
R_MN = M @ N
print("Matrix Multiplication:\n", np.around(R_MN, decimals=4))

Matrix Multiplication:
 [[-0.945 -0.239  0.224]
 [-0.14   0.913  0.384]
 [-0.296  0.332 -0.896]]


In [19]:
q_M = rotation_to_quaternion(M)
print("Qm:\n", q_M)

q_N = rotation_to_quaternion(N)
print("Qn:\n", q_N)

Qm:
 [0.794 0.01  0.593 0.131]
Qn:
 [ 0.704 -0.062  0.704  0.062]


In [20]:
q_MN = np.around(multiply_quaternions(q_M, q_N), decimals=5)
print("Qm x Qn:\n", q_MN)

Qm x Qn:
 [ 0.134 -0.098  0.969  0.185]


In [21]:
rotation_matrix = quaternion_to_rotation(q_MN)
print("Rot(Qm x Qn)\n", np.around(rotation_matrix, decimals=5))

print("Rot(Qm x Qn) is same as Original Rotation Matrix Multiplication ")

Rot(Qm x Qn)
 [[-0.945 -0.239  0.224]
 [-0.14   0.913  0.384]
 [-0.296  0.332 -0.896]]
Rot(Qm x Qn) is same as Original Rotation Matrix Multiplication 


In [22]:
## Verification
r = Rotation.from_matrix(R_MN)
scipy_quat = r.as_quat()[[-1, 0, 1, 2]]
print("Scipy Result, Rot => Quat: ", scipy_quat)
print("Result is same as Qm x Qn")

Scipy Result, Rot => Quat:  [ 0.134 -0.098  0.969  0.185]
Result is same as Qm x Qn


<font color='blue' size=4.0/> 2.b.4 Try to interpolate any 3D model between two rotation matrices and visualize

In [23]:
# Interpolate between Rotation Matrices
def interpolate_rotations(R0, R1, steps=5):
    
    q0 = rotation_to_quaternion(R0)
    q1 = rotation_to_quaternion(R1)
    
    quaternions = [None] * (steps + 2)
    quaternions[0] = q0
    quaternions[-1] = q1

    dot = np.dot(q0, q1)
    
    if dot < 0.0:
        q0 = -q0
        dot = -dot

    theta_ = np.arccos(dot)
    sin_theta_ = np.sin(theta_)
    
    step_size = 1/(steps + 1)
    
    for i in range(1, steps + 1):
        
        amount = i * step_size
        
        theta = theta_ * amount
        sin_theta = np.sin(theta)

        s0 = np.cos(theta) - dot * sin_theta / sin_theta_
        s1 = sin_theta / sin_theta_
        qr = s0 * q0 + s1 * q1
        qr = normalize(qr)
        
        quaternions[i] = qr
    
    rotations = []
    for i in range(1, len(quaternions)):
        quat_diff = multiply_quaternions(quaternion_conjugate(quaternions[i - 1]), quaternions[i])
        quat_diff = normalize(quat_diff)
        R = quaternion_to_rotation(quat_diff)
        rotations.append(R)
    return rotations

In [24]:
R0 = Rotation.from_euler('x', 90, degrees=True).as_matrix()
R1 = Rotation.from_euler('y', 90, degrees=True).as_matrix()

rotations = interpolate_rotations(R0, R1)

## Verifying whether final matrices are smae
R_ = R0
for rotation in rotations:
    R_ = R_ @ rotation

print(R_)
print(R1)

[[ 0.  0.  1.]
 [ 0.  1. -0.]
 [-1.  0.  0.]]
[[ 0.  0.  1.]
 [ 0.  1.  0.]
 [-1.  0.  0.]]


In [25]:
## Animation
point_cloud = o3d.io.read_point_cloud('./data/bunny.ply')
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.03, origin=np.array([0., 0., 0.]))

point_cloud.rotate(R=R0)
mesh_frame.rotate(R=R0)

vis = o3d.visualization.Visualizer()
vis.create_window()

vis.add_geometry(mesh_frame)
vis.add_geometry(point_cloud)

for rotation in rotations:
    time.sleep(0.9)
    point_cloud.rotate(R=rotation)
    mesh_frame.rotate(R=rotation)

    vis.update_geometry(point_cloud)
    vis.update_geometry(mesh_frame)
    
    vis.poll_events()
    vis.update_renderer()
    
vis.destroy_window()

In [26]:
## Verification
point_cloud = o3d.io.read_point_cloud('./data/bunny.ply')
point_cloud.paint_uniform_color([1, 0.5, 0.])

mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.03, origin=np.array([0., 0., 0.]))

o3d.visualization.draw([point_cloud, mesh_frame])

point_cloud.rotate(R=R0)
mesh_frame.rotate(R=R0)
o3d.visualization.draw([point_cloud, mesh_frame])

point_cloud.rotate(R=R1)
mesh_frame.rotate(R=R1)
o3d.visualization.draw([point_cloud, mesh_frame])

## c) Exponential maps (Bonus)

1. What is the idea behind exponential map representation of rotation matrices?
2. Perform matrix exponentiation and obtain the rotation matrix to rotate a vector $P$ around $\omega$ for $\theta$ seconds.
$$
\omega = \begin{bmatrix}2 \\ 1 \\ 15 \end{bmatrix}
$$

$$
\theta = 4.1364
$$

3. Compute the logarithmic map (SO(3) to so(3)) of the rotation matrix to obtain the rotation vector and the angle of rotation
$$
\begin{bmatrix}
0.1 &  -0.9487 & 0.3 \\
0.9487 & 0.  & -0.3162 \\
0.3   &  0.3162  & 0.9 
\end{bmatrix}
$$
You can use inbuilt libraries **only to verify** your results.

<font color='blue' size=4.0/> 2.c.1 Idea behind exponential map representation of rotation matrices

Parameterizing rotation matricies as 3x3 rotation matricies and quaternions is problamatic as certain constraint have to be followed during the updates. Exponential Map representation solve this problem by representing Rotation matrices with only three parameters, a free vector whose magnitude represents the degree of rotation. In this representation no constraints have to be followed so paramters updates are straightforward.

<font color='blue' size=4.0/> 2.c.2 Perform matrix exponentiation and obtain the rotation matrix to rotate a vector 𝑃 around 𝜔 for 𝜃 seconds

In [27]:
def get_exponential_rotation(w, theta):
    w_mag = np.linalg.norm(w, 2)
    
    ## Skey symmetric Matrix
    w_hat = np.matrix([[0, -w[2], w[1]],
                       [w[2], 0, -w[0]],
                       [-w[1], w[0], 0]])
    
    ## Using Rodrigues Formula
    rot_mat = np.eye(3) + (np.sin(w_mag*theta)/w_mag)*w_hat + ((1 - np.cos(w_mag*theta))/w_mag**2)*w_hat*w_hat
    return rot_mat

In [28]:
w = np.array([2, 1, 15])
theta = 4.1364

exp_rot = get_exponential_rotation(w, theta)
print(exp_rot)

[[ 0.995  0.099 -0.006]
 [-0.099  0.995  0.014]
 [ 0.007 -0.013  1.   ]]


In [29]:
## Verification
exp_rot_ = Rotation.from_rotvec(w*theta).as_matrix()
print(exp_rot_)

[[ 0.995  0.099 -0.006]
 [-0.099  0.995  0.014]
 [ 0.007 -0.013  1.   ]]


<font color='blue' size=4.0/> 2.c.3 Compute the logarithmic map (SO(3) to so(3)) of the rotation matrix to obtain the rotation vector and the angle of rotation

In [30]:
def get_logarithmic_map(R):
    ## R = I + sin(t)K + ((1 - cos(t))K^2
    ## K is skew_symmetric => trace(K) = 0
    
    tr = np.trace(R)
    
    t = np.arccos((tr - 1)/2)
    K = (R - R.T)/2*np.sin(t)
    
    K_hat = np.array([K[2, 1], K[0, 2], K[1, 0]])
    
    return K_hat, t

In [31]:
R = np.matrix([[0.1, -0.9487, 0.3],
               [0.9487, 0., -0.3162],
               [0.3, 0.3162, 0.9]])

w, t = get_logarithmic_map(R)
print(f"Rotation vector: {w}, Rotation angle: {t}")

Rotation vector: [0.316 0.    0.949], Rotation angle: 1.5707963267948966


In [32]:
## Verification
w_ = Rotation.from_matrix(R).as_rotvec()
print("Verify Rotation Angle: ", np.linalg.norm(w_))

w_ = normalize(w_)
print("Verify Rotation vector: ", w_)

Verify Rotation Angle:  1.570803391744983
Verify Rotation vector:  [0.316 0.    0.949]


# 3. Data representations

## a) Octomaps

1. Why is an Octomap memory efficient?
2. When do we update an Octomap and why?
3. When would you likely use an octomap instead of a point cloud?
 

<font color='blue' size=4.0/> 3.a.1 Why is an Octomap memory efficient?

Octomap is based on heirarchical data structure called octree. In Octree, each node represents a 3D cubic volume called voxel. Octomaps discretize the 3D space into voxels, which can be further divided depending on the occupancy of space. Specifically, for free and fully occupied spaces sub-division is not required and for partially occupied space, voxels can be sub-divided until minimum voxel size is reached. This type of heirarchical 3D mapping allow Octomap to be memory efficient for storing data. Additionally, data access complexity is also reduced owing to tree-like hierarchy.

<font color='blue' size=4.0/> 3.a.2 When do we update an Octomap and why?

Octomap is updated when subsequent observations measure a different state of voxel than that measured by previous observations. Updates are done to account for the dynamic changes in the 3D space.

<font color='blue' size=4.0/> 3.a.3 When would you likely use an octomap instead of a point cloud?

For navigation tasks Octomap is more suitable than a point cloud. Point Clouds store large amount of measurement points and do not differentiate between free and unmapped areas. Further, the motion of vehicle introduces noise that can result in inaccurate mapping of 3D environment. Octomaps don't suffer from these issues, as they model free space explicitly and probabilistic mapping allow them to account for uncertainity introduced by noise.

## b) Signed Distance Functions

1. How do we determine object surfaces using SDF?
2. How do we aggregate views from multiple cameras? (just a general overview is fine)
3. Which preserves details better? Voxels or SDF? Why?
4. What’s an advantage of SDF over a point cloud?


<font color='blue' size=4.0/> 3.b.1 How do we determine object surfaces using SDF?

Signed distance functions are used to calculate the distance of a point from surface of an object and sign is used for determining whether the point is inside the surface or outside. A ray is cast from a source point towards the object, SDF is evaluated for points (voxels) intersecting the ray. Surface can be approximated by interpolating bewteen voxels having opposite signs for SDF value.

<font color='blue' size=4.0/> 3.b.2 How do we aggregate views from multiple cameras?

Voxel values are updated with weighted average of SDF values for multiple camera views. Weight is directly proportional to the number of measurements for voxel and represents the confidence in the measurement.

<font color='blue' size=4.0/> 3.b.3 Which preserves details better? Voxels or SDF? Why?

SDF preserve details better because they are build on top of voxels. They contain all advantages of voxels, additionally each voxel in SDF also contains the distance to the nearest obstacle and whether the voxel is inside the surface or outside it, making them readily amenable for planning task.

<font color='blue' size=4.0/> 3.b.4 What’s an advantage of SDF over a point cloud?

Point clouds are not suitable for describing surfaces and don't explicitly contain information about the nearest obstace for each point cloud. SDF on the other hand can be readily used to describe surface by it's implicit representation. Additionally, SDF encodes the distance of point from the surface and whether it's inside or outside. Finally, the surfaces given by SDF are very smooth compared to point cloud.

# References and Resources

1. Gimbal locks and quaternions: https://youtu.be/YF5ZUlKxSgE
2. Exponential map: 
    1. 3 Blue 1 Brown: https://youtu.be/O85OWBJ2ayo
    2. Northwestern Robotics: https://youtu.be/v_KBHaG0mas
3. Bunny ply is taken from: http://graphics.im.ntu.edu.tw/~robin/courses/cg03/model/